<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#All-packages-import" data-toc-modified-id="All-packages-import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>All packages import</a></span></li><li><span><a href="#Enable-GPU-if-avalaible" data-toc-modified-id="Enable-GPU-if-avalaible-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Enable GPU if avalaible</a></span></li><li><span><a href="#Preprocess-a-RTE-report" data-toc-modified-id="Preprocess-a-RTE-report-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preprocess a RTE report</a></span><ul class="toc-item"><li><span><a href="#Function-to-extract-text-of-pdf" data-toc-modified-id="Function-to-extract-text-of-pdf-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Function to extract text of pdf</a></span></li><li><span><a href="#Import-text-and-process-it" data-toc-modified-id="Import-text-and-process-it-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Import text and process it</a></span></li><li><span><a href="#Save-on-json-format-compatible-with-the-Q&amp;A-pipeline" data-toc-modified-id="Save-on-json-format-compatible-with-the-Q&amp;A-pipeline-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Save on json format compatible with the Q&amp;A pipeline</a></span></li></ul></li><li><span><a href="#Import-and-use-a-CamemBERT-Q&amp;A-model" data-toc-modified-id="Import-and-use-a-CamemBERT-Q&amp;A-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Import and use a CamemBERT Q&amp;A model</a></span><ul class="toc-item"><li><span><a href="#Import-the-model-and-create-the-pipe" data-toc-modified-id="Import-the-model-and-create-the-pipe-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Import the model and create the pipe</a></span></li><li><span><a href="#Use-the-pipe-to-answer-question" data-toc-modified-id="Use-the-pipe-to-answer-question-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Use the pipe to answer question</a></span></li></ul></li><li><span><a href="#Fine-tune-CamemBERT-Q&amp;A-for-RTE-report" data-toc-modified-id="Fine-tune-CamemBERT-Q&amp;A-for-RTE-report-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Fine-tune CamemBERT Q&amp;A for RTE report</a></span></li><li><span><a href="#Use-our-fine-tuned-RTE-model" data-toc-modified-id="Use-our-fine-tuned-RTE-model-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Use our fine-tuned RTE model</a></span></li><li><span><a href="#Evaluate-Q&amp;A-models" data-toc-modified-id="Evaluate-Q&amp;A-models-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Evaluate Q&amp;A models</a></span></li><li><span><a href="#Long-form-Q&amp;A-model-:-does-it-improve-performances-?" data-toc-modified-id="Long-form-Q&amp;A-model-:-does-it-improve-performances-?-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Long-form Q&amp;A model : does it improve performances ?</a></span></li><li><span><a href="#Generate-Questions" data-toc-modified-id="Generate-Questions-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Generate Questions</a></span></li></ul></div>

# All packages import

In [ ]:
from transformers import pipeline
import os
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LAParams
from io import StringIO
from pdfminer.converter import TextConverter
from collections import Counter
import json

In [ ]:
!pip install tensorflow

In [ ]:
!pip install --upgrade pip

     |████████████████████████████████| 2.1 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-mjy98a2f/farm-haystack_9d656512fba24fc68ba8e2edc0382841
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-install-mjy98a2f/farm-haystack_9d656512fba24fc68ba8e2edc0382841
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 321 kB 3.9 MB/s 
     |████████████████████████████████| 3.3 MB 30.2 MB/s 
     |████████████████████████████████| 10.7 MB 25.1 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 79 kB 8.2 MB/s 
     |████████████████████████████████| 5.6 MB 32.5 MB/s 
     |████████████████████████████████| 14.1 MB 28.9 MB/s 
     |████████████████████████████████| 359 kB 14.4 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 981 kB 34.0 MB/s 
     |█████████████████

In [ ]:
!pip install transformers

# Enable GPU if avalaible

In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Sat Mar 19 10:00:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Preprocess a RTE report 

This section is a first pre-processing of a text. The process has several errors and the text must be processed "by hand" next. 

## Function to extract text of pdf

In [ ]:
def extract_txt(pdf_doc):
    """
    Fonction d'extraction de texte à partir d'un pdf
    """
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams) # codec=codec,
    interpreter = PDFPageInterpreter(rsrcmgr, device)   
    with open(pdf_doc, 'rb') as fp:
        num_pages=0
        for page in PDFPage.get_pages(fp):
            num_pages+=1
            interpreter.process_page(page)    
        text = retstr.getvalue()
        num_words = sum(Counter(text.split()).values())
        method = "pdfminer"
    
    return text, num_pages, num_words, method

## Import text and process it

In [ ]:
path_rapport = "Futurs-Energetiques-2050-principaux-resultats.pdf"

In [ ]:
# store result in a dict
rapport = {}
rapport["text"], rapport["num_pages"], rapport["num_words"], rapport["method"] = extract_txt(path_rapport)
# re-process text to remove the layout of the pdf 
rapport["text"] = rapport["text"].replace("\n\n", "|||||").replace("\n", " ").replace("|||||", "\n\n")

## Save on json format compatible with the Q&A pipeline

In [ ]:
dicts = []
# dicts.append({'content': rapport["text"], "meta": {"name":"Futurs-Energetiques-2050-principaux-resultats"}})
for txt in rapport["text"].split("\n\n"):
    dicts.append({'text': txt, "meta": {"name":"Futurs-Energetiques-2050-principaux-resultats"}})

In [ ]:
with open("/content/drive/MyDrive/Projet Option - ChatBot/0 - Biblio/rapport_rte_v2.json", "w") as outfile:
    json.dump(dicts, outfile)

In [ ]:
import json
all_lines = []
with open('/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/1-33.txt') as f:
    lines = f.readlines()
    all_lines.append(lines)
with open("/content/drive/MyDrive/Projet Option - ChatBot/0 - Biblio/rapport_rte_34_end.txt") as f:
  # print(f.readlines())
  all_lines.append(f.readlines())



In [ ]:
all_lines[1]

In [ ]:
all_lines[0]
data_txt = []
for line in all_lines[0]+all_lines[1]: 
  line = line.rstrip()
  if line not in ["\n", ""]:
    data_txt.append(line)
print(len(data_txt))

323


In [ ]:
dicts = []
for txt in data_txt:
    dicts.append({'content': txt, "meta": {"name":"Futurs-Energetiques-2050-principaux-resultats"}})
dicts = {"data": dicts}

In [ ]:
with open("/content/drive/MyDrive/Projet Option - ChatBot/0 - Biblio/rapport_rte_v2.json", "w") as outfile:
    json.dump(dicts, outfile, ensure_ascii=False)

# Import and use a CamemBERT Q&A model

## Import the model and create the pipe

In [ ]:
report_path = "/content/drive/MyDrive/Projet Option - ChatBot/0 - Biblio/rapport_rte_v1.json"

# Import text 
with open(report_path) as json_file:
    dict_rapport = json.load(json_file)# ["data"]
print(len(dict_rapport))

1


In [ ]:
import json
from haystack.utils import convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import ElasticsearchRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from pathlib import Path
import pickle
from time import sleep


# #############################################
# ## Launching and setting up pipe obj       ##
# #############################################
from haystack.utils import launch_es

! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")
report_path = "/content/drive/MyDrive/Projet Option - ChatBot/0 - Biblio/rapport_rte_v2.json"

# Import text 
with open(report_path) as json_file:
    dict_rapport = json.load(json_file)["data"]

# write document on ElasticSearch server
document_store.write_documents(dict_rapport)

## Initalize Retriever, Reader,  & Pipeline
retriever = ElasticsearchRetriever(document_store=document_store)
reader = FARMReader(model_name_or_path="etalab-ia/camembert-base-squadFR-fquad-piaf", use_gpu=True)

pipe = ExtractiveQAPipeline(reader, retriever)


INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find etalab-ia/camembert-base-squadFR-fquad-piaf locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/515 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Automatically detected language from language model name: french
INFO - haystack.modeling.model.language_model -  Loaded etalab-ia/camembert-base-squadFR-fquad-piaf


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


## Use the pipe to answer question

In [ ]:
# Example of use of this pipe 
# Get the prediction for a question example 
prediction = pipe.run(
    query="Quelle diminution la SNBC prévoit pour la consommation énergétique d’ici 2050 ?", 
    params={"Reader": {"top_k": 3}}
)

# print the result 
print_answers(prediction, details="medium")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 18.34 Batches/s]


Query: Quelle diminution la SNBC prévoit pour la consommation énergétique d’ici 2050 ?
Answers:
[   {   'answer': '40 %',
        'context': 'elle prévoit que la consommation d’énergie finale de la '
                   'France diminue de 40 % en trente ans. Il s’agit d’une '
                   'ambition très forte, dans le haut de la ',
        'score': 0.9772907197475433},
    {   'answer': '15 %',
        'context': 'rs énergétiques 2050 » chiffrent le gain atteignable à '
                   'environ 90 TWh d’électricité, soit une réduction de 15 % '
                   '(en plus de l’efficacité énergétique).',
        'score': 0.9584720432758331},
    {   'answer': '40 %',
        'context': 'isant la consommation d’énergie finale. La SNBC prévoit '
                   'une diminution de 40 % de la consommation énergétique '
                   'd’ici 2050, ce qui constitue une cible t',
        'score': 0.9558982849121094}]


# Fine-tune CamemBERT Q&A for RTE report

Exemple de json pour l'entrainement d'un modèle 

In [ ]:
# from datasets import load_dataset
# squad = load_dataset("squad")
# squad["train"][0] 
{
    'answers': {
        'answer_start': [515],
        'text': ['Saint Bernadette Soubirous']
    },
    'context':
    'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
    'id': '5733be284776f41900661182',
    'question':
    'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
    'title': 'University_of_Notre_Dame'
}

{
    'context': "",
    'question':"",
    'answers':""
}

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

In [ ]:
import json
from haystack.utils import convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import ElasticsearchRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from pathlib import Path
import pickle
from time import sleep

reader = FARMReader(
    model_name_or_path="etalab-ia/camembert-base-squadFR-fquad-piaf", use_gpu=True)
# data_dir = "data/squad20"
data_dir = "/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/Question-Réponse/"
reader.train(data_dir=data_dir,
             train_filename="data_train_squad_format.json",
             use_gpu=True,
             n_epochs=4,
             save_dir="camembertV2")

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find etalab-ia/camembert-base-squadFR-fquad-piaf locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Automatically detected language from language model name: french
INFO - haystack.modeling.model.language_model -  Loaded etalab-ia/camembert-base-squadFR-fquad-piaf
INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystac

In [ ]:
# Saving the model happens automatically at the end of training into the `save_dir` you specified
# However, you could also save a reader manually again via:
# reader.save(directory="my_model")

# Use our fine-tuned RTE model 

In [ ]:
import json
from haystack.utils import convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import ElasticsearchRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from pathlib import Path
import pickle
from time import sleep


# #############################################
# ## Launching and setting up pipe obj       ##
# #############################################
from haystack.utils import launch_es

! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30


# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")
report_path = "/content/drive/MyDrive/Projet Option - ChatBot/0 - Biblio/rapport_rte_v1.json"

# Import text 
with open(report_path) as json_file:
    dict_rapport = json.load(json_file)

# write document on ElasticSearch server
document_store.write_documents(dict_rapport)

## Initalize Retriever, Reader,  & Pipeline
retriever = ElasticsearchRetriever(document_store=document_store)
reader = FARMReader(model_name_or_path="/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/camembertV2", use_gpu=True)

pipe = ExtractiveQAPipeline(reader, retriever)


INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at /content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/camembertV2
INFO - haystack.modeling.model.language_model -  Loaded /content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/camembertV2
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": true, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"

In [ ]:
# Example of use of this pipe 
# Get the prediction for a question example 
prediction = pipe.run(
    query="Quelle diminution la SNBC prévoit pour la consommation énergétique d’ici 2050 ?", 
    params={"Reader": {"top_k": 3}}
)

# print the result 
print_answers(prediction, details="medium")

Inferencing Samples:   0%|          | 0/6 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 6/6 [00:08<00:00,  1.39s/ Batches]


Query: Quelle diminution la SNBC prévoit pour la consommation énergétique d’ici 2050 ?
Answers:
[   {   'answer': '15 %',
        'context': 'n  atteignable  à  environ  90  TWh  d’électricité, soit '
                   'une réduction de 15 % (en plus de  l’efficacité '
                   'énergétique). \n'
                   '\n'
                   'Y  parvenir  nécessite  d’all',
        'score': 0.900974690914154},
    {   'answer': '40 %',
        'context': 'ant  la consommation d’énergie finale. La SNBC prévoit  '
                   'une diminution de 40 % de la consommation éner- gétique  '
                   'd’ici  2050,  ce  qui  constitue  une',
        'score': 0.6947499811649323},
    {   'answer': '~40 MtCO2eq/an',
        'context': ' MtCO2eq/an  entre 2020 et 2030, ~30 MtCO2eq/an entre 2030 '
                   'et  2040, ~40 MtCO2eq/an entre 2040 et 2050.\n'
                   '\n'
                   'La France peut en effet tirer parti d’un mix ',
        'score': 0.

# Evaluate Q&A models 

In [ ]:
import json 

# Opening JSON file
f = open("/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/Question-Réponse/data_test_squad_format.json")
 
# returns JSON object as
# a dictionary
data_test = json.load(f)


In [ ]:
from haystack.schema import EvaluationResult, MultiLabel, Label, Answer, Document

# We can load evaluation labels from the document store
# We are also opting to filter out no_answer samples
# eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=False)
# eval_labels = [label for label in eval_labels if not label.no_answer]  # filter out no_answer cases

## Alternative: Define queries and labels directly

eval_labels = [
   MultiLabel(
       labels=[
           Label(
               query=data_test["data"][i]["paragraphs"][0]['qas'][0]['question'],
               answer=Answer(
                   answer=data_test["data"][i]["paragraphs"][0]['qas'][0]['answers'][0]['text'],
               ),
               document=Document(
                   id=data_test["data"][i]["paragraphs"][0]['document_id'],
                   content_type="text",
                   content=data_test["data"][i]["paragraphs"][0]['context']
               ),
               is_correct_answer=True,
               is_correct_document=True,
               origin="gold-label"
           )
       ]
   ) for i in range(len(data_test["data"]))
]


# Similar to pipeline.run() we can execute pipeline.eval()
eval_result = pipe.eval(labels=eval_labels, params={"Retriever": {"top_k": 3}})


Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 24.46 Batches/s]


In [ ]:
reader_result = eval_result["Reader"]
reader_result.head()


,multilabel_id,query,filters,gold_answers,answer,context,exact_match,f1,rank,document_id,gold_document_ids,offsets_in_document,gold_offsets_in_documents,type,node,eval_mode
0,154047218475213277,Que sous-entendent les scénarios 100% renouvelables ou de prolongation des c...,b'null',[grand nombre de prérequis techniques critiques soient respectés à court terme],qu’un grand nombre de prérequis techniques critiques soient respectés à cour...,tuels au-delà de 60 ans impliquent qu’un grand nombre de prérequis technique...,0.0,0.956522,1.0,312295387148fc8fa6015cdae4e1c14,[15],"[{'start': 240, 'end': 323}]",[],answer,Reader,integrated
1,154047218475213277,Que sous-entendent les scénarios 100% renouvelables ou de prolongation des c...,b'null',[grand nombre de prérequis techniques critiques soient respectés à court terme],paris technologiques lourds pour être au rendez-vous de la neutralité carbone,"réacteurs nucléaires existants au-delà de 60 ans, impliquent des paris techn...",0.0,0.090909,2.0,b4431cd74a2b7f1e2ffe3d3c9638f213,[15],"[{'start': 165, 'end': 242}]",[],answer,Reader,integrated
2,154047218475213277,Que sous-entendent les scénarios 100% renouvelables ou de prolongation des c...,b'null',[grand nombre de prérequis techniques critiques soient respectés à court terme],exploitation de certains des réacteurs actuels,rios des «Futurs énergétiques 2050» reposent sur une exploitation de certain...,0.0,0.117647,3.0,8dd9975d24d0b368950611dbe67cf30a,[15],"[{'start': 70, 'end': 116}]",[],answer,Reader,integrated
3,154047218475213277,Que sous-entendent les scénarios 100% renouvelables ou de prolongation des c...,b'null',[grand nombre de prérequis techniques critiques soient respectés à court terme],pas tributaire de la prolongation des réacteurs au-delà de 60 ans et sa traj...,pas tributaire de la prolongation des réacteurs au-delà de 60 ans et sa tra...,0.0,0.055556,4.0,a9e29da3f1d1b2b335d20da2f7e747fa,[15],"[{'start': 282, 'end': 434}]",[],answer,Reader,integrated
4,154047218475213277,Que sous-entendent les scénarios 100% renouvelables ou de prolongation des c...,b'null',[grand nombre de prérequis techniques critiques soient respectés à court terme],prolonger certains au-delà de 60 ans,"ation de l’essentiel des réacteurs jusqu’à 60 ans, (2) en prolonger certains...",0.0,0.117647,5.0,fcd7897e6a2bcc10049200b69e81149e,[15],"[{'start': 281, 'end': 317}]",[],answer,Reader,integrated


In [ ]:
reader_result.to_csv("/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/eval_Camembert.csv")

In [ ]:
import numpy as np
all_f1 = []
for i in range(len(reader_result)//3):
    f1 = max(reader_result["f1"].iloc[3*i], reader_result["f1"].iloc[3*i+1], reader_result["f1"].iloc[3*i+2])
    all_f1.append(f1)
np.mean(all_f1)

0.353127711724928

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importer les questions/réponses

In [ ]:
# q_a_dataset = ...

In [ ]:
import json
data_path = [
        "/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/Question-Réponse/q&a_40_52.json",
        "/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/Question-Réponse/q&a_rte_part3.json",
        "/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/Question-Réponse/qa_rte_Thibaud.json"
]

# q_a_dataset = {}
 
# # Opening JSON file
# for path in data_path:
#   with open(path) as json_file:
#       data = json.load(json_file)
#       q_a_dataset

In [ ]:
len(q_a_dataset)

173

In [ ]:
{
    'answers': {
        'answer_start': [515],
        'text': ['Saint Bernadette Soubirous']
    },
    'context':
    'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
    'id': '5733be284776f41900661182',
    'question':
    'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
    'title': 'University_of_Notre_Dame'
}

In [ ]:
q_a_dataset[1]['context'].find(q_a_dataset[1]['answers']), q_a_dataset[47]

(208,
 {'answers': 'remplacer les produits pétroliers (essence et gasoil) par de l’électricité (ou par de l’hydrogène lui-même produit à partir d’électricité) dans les véhicules particuliers et les poids lourds, et à remplacer le fioul et le gaz fossile pour le chauffage',
  'context': 'Les actions ayant le meilleur impact climatique consistent à remplacer les produits pétroliers (essence et gasoil) par de l’électricité (ou par de l’hydrogène lui-même produit à partir d’électricité) dans les véhicules particuliers et les poids lourds, et à remplacer le fioul et le gaz fossile pour le chauffage dans le cadre de la rénovation des bâtiments. Dans l’industrie, le développement de l’électricité ou de l’hydrogène bas-carbone dans certains procédés ou encore le recours aux chaudières électriques permet également de réduire les émissions.',
  'qusetion': 'Comment réduire les émissions de la France dûes aux produits pétroliers ?'})

In [ ]:
from tqdm.notebook import tqdm
dataset_formated = []
for i, q_a in tqdm(enumerate(q_a_dataset)):
  
  temp_dict = {
      'answers': {
          'answer_start': [q_a['context'].find(q_a['answers'])],
          'text': [q_a["answers"]]
      },
      'context': q_a["context"],
      'id': i,
      'question': q_a["question"],
      'title': 'RTE_rapport'
  }
  dataset_formated.append(temp_dict)


0it [00:00, ?it/s]

In [ ]:
dataset_formated[0]

{'answers': {'answer_start': [0],
  'text': ['L’hydrogène constitue en premier lieu un moyen de décarboner des secteurs difficiles voire impossibles à électrifier']},
 'context': 'L’hydrogène constitue en premier lieu un moyen de décarboner des secteurs difficiles voire impossibles à électrifier sur le plan technique ou économique. À ce titre, l’action prioritaire à engager consiste à remplacer l’hydrogène fossile (issu à 95% du gaz, pétrole et charbon) actuellement utilisé par l’industrie dans ses process en France (dans le secteur du raffinage, de la production d’ammoniac ou de la chimie) par de l’hydrogène bas-carbone',
 'id': 0,
 'question': "Quel levier l'hydrogène constitue-t-il ?",
 'title': 'RTE_rapport'}

In [ ]:
len(dataset_formated)

173

In [ ]:
!pip install sklearn

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test, _, _ = train_test_split(dataset_formated, dataset_formated, test_size=60)
data_train[0]

{'answers': {'answer_start': [52],
  'text': ['les scénarios comprenant un parc nucléaire de 40 GW au moins (N2 et N03) peuvent conduire, à long terme, à des coûts plus bas pour la collectivité qu’un scénario 100 % renouvelable reposant sur de grands parcs.']},
 'context': 'L’étude conclut avec un bon niveau de confiance que les scénarios comprenant un parc nucléaire de 40 GW au moins (N2 et N03) peuvent conduire, à long terme, à des coûts plus bas pour la collectivité qu’un scénario 100 % renouvelable reposant sur de grands parcs. Ceci est vrai même si les coûts « bruts » de production des nouvelles centrales nucléaires sont en moyenne plus élevés que ceux associés aux grands parcs d’énergies renouvelables. En effet, l’intégration de volumes importants d’éoliennes ou de panneaux solaires engendre de très importants besoins en flexibilités (stockage, pilotage de la demande et nouvelles centrales d’appoint) pour pallier leur variabilité, ainsi que des renforcements des réseaux (raccorde

In [ ]:
with open('/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/Question-Réponse/data_train.json', 'w') as fout:
    json.dump(data_train , fout)

In [ ]:
with open('/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/Question-Réponse/data_test.json', 'w') as fout:
    json.dump(data_test , fout)

In [ ]:
len(data_train)

113

In [ ]:
with open('/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/Question-Réponse/q&a_rte_Loulou.json', 'w') as fout:
    json.dump(dataset_formated , fout)

In [ ]:
from datasets import load_dataset
squad = load_dataset("squad")
squad["train"][0] 

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

# Generate Questions

In [ ]:
!pip install farm-haystack

In [ ]:
# Imports needed to run this notebook

from pprint import pprint
from tqdm import tqdm
from haystack.nodes import QuestionGenerator, ElasticsearchRetriever, FARMReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import (
    QuestionGenerationPipeline,
    RetrieverQuestionGenerationPipeline,
    QuestionAnswerGenerationPipeline,
)
from haystack.utils import launch_es, print_questions

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/


In [ ]:
# Option 2: In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
text1 = "En France, environ 60% de l’énergie utilisée est d’origine fossile : il s’agit principalement des produits pétroliers (de l’ordre de 40 %), du gaz naturel (de l’ordre de 20 %) et du charbon (moins de 1 %). Cette énergie dépend des importations des pays producteurs (notamment l’Arabie saoudite, le Kazakhstan, la Russie, le Nigeria et l’Algérie pour le pétrole brut, la Norvège, la Russie, les Pays-Bas et le Nigeria pour le gaz). Les crises énergétiques qui se succèdent ont montré combien la France était exposée par ce biais aux variations des cours des produits énergétiques sur les marchés mondiaux, qui dépendent de dynamiques géopolitiques complexes et de l’état de l’économie mondiale."
text2 = "C’est pourtant bien ce système articulé autour des énergies fossiles qui a été le socle de la croissance économique du pays au cours des Trente Glorieuses. Malgré les chocs pétroliers, il a alimenté la France avec une énergie bon marché, encore abondante et facilement stockable. Les combustibles fossiles satisfont aujourd’hui une consommation finale de plus de 930 TWh par an, contre 430 TWh pour l’électricité. Le système électrique français, contrairement à celui de la majorité de ses voisins, n’est pas assis sur les énergies fossiles. Sa caractéristique principale est de reposer en majorité sur un parc de 56 réacteurs nucléaires, construits et mis en service de manière très rapprochée entre la fin des années 1970 et le début des années 1990 pour la plupart, et qui se sont ajoutés à une base de production hydraulique déjà importante (60 TWh). Le programme électronucléaire français répondait à un souci d’autonomie énergétique à la suite des chocs pétroliers. Aujourd’hui, il n’est pas contestable qu’il constitue un atout majeur de la France dans la lutte contre le changement climatique en produisant une électricité très largement décarbonée en grandes quantités."
text3 = "Comme dans toutes les démocraties occidentales, le choix du nucléaire civil suscite en France un débat démocratique. La discussion s’est cristallisée, ces dernières années, autour de la notion de «part» du nucléaire dans le mix électrique, à tel point qu’une partie de la population a pu croire que cette part renvoyait à celle du nucléaire dans la consommation énergétique totale de la France. Or, si le nucléaire représente bien 70% de l’électricité produite en France, il représente moins de 20% de l’énergie finale utilisée par les français. La prépondérance du nucléaire dans la production d’électricité ne doit pas occulter la dépendance de la France aux énergies fossiles et importées pour ses besoins en énergie. Dès lors, l’atteinte de la neutralité carbone oblige à renoncer en quasi-totalité à ces énergies fossiles."

docs = [{"content": text1}, {"content": text2}, {"content": text3}]

# Initialize document store and write in the documents
document_store = ElasticsearchDocumentStore()
document_store.write_documents(docs)

# Initialize Question Generator
question_generator = QuestionGenerator()

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/195 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
reader = FARMReader("deepset/roberta-base-squad2")
qag_pipeline = QuestionAnswerGenerationPipeline(question_generator, reader)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0    0 
INFO - haystack.modeling.infer -  /w\  /w\
INFO - haystack.modeling.infer -  /'\  / \


In [ ]:
# Fill the document store with a French document.
text1 = "En France, environ 60% de l’énergie utilisée est d’origine fossile : il s’agit principalement des produits pétroliers (de l’ordre de 40 %), du gaz naturel (de l’ordre de 20 %) et du charbon (moins de 1 %). Cette énergie dépend des importations des pays producteurs (notamment l’Arabie saoudite, le Kazakhstan, la Russie, le Nigeria et l’Algérie pour le pétrole brut, la Norvège, la Russie, les Pays-Bas et le Nigeria pour le gaz). Les crises énergétiques qui se succèdent ont montré combien la France était exposée par ce biais aux variations des cours des produits énergétiques sur les marchés mondiaux, qui dépendent de dynamiques géopolitiques complexes et de l’état de l’économie mondiale."
text2 = "C’est pourtant bien ce système articulé autour des énergies fossiles qui a été le socle de la croissance économique du pays au cours des Trente Glorieuses. Malgré les chocs pétroliers, il a alimenté la France avec une énergie bon marché, encore abondante et facilement stockable. Les combustibles fossiles satisfont aujourd’hui une consommation finale de plus de 930 TWh par an, contre 430 TWh pour l’électricité. Le système électrique français, contrairement à celui de la majorité de ses voisins, n’est pas assis sur les énergies fossiles. Sa caractéristique principale est de reposer en majorité sur un parc de 56 réacteurs nucléaires, construits et mis en service de manière très rapprochée entre la fin des années 1970 et le début des années 1990 pour la plupart, et qui se sont ajoutés à une base de production hydraulique déjà importante (60 TWh). Le programme électronucléaire français répondait à un souci d’autonomie énergétique à la suite des chocs pétroliers. Aujourd’hui, il n’est pas contestable qu’il constitue un atout majeur de la France dans la lutte contre le changement climatique en produisant une électricité très largement décarbonée en grandes quantités."
text3 = "Comme dans toutes les démocraties occidentales, le choix du nucléaire civil suscite en France un débat démocratique. La discussion s’est cristallisée, ces dernières années, autour de la notion de «part» du nucléaire dans le mix électrique, à tel point qu’une partie de la population a pu croire que cette part renvoyait à celle du nucléaire dans la consommation énergétique totale de la France. Or, si le nucléaire représente bien 70% de l’électricité produite en France, il représente moins de 20% de l’énergie finale utilisée par les français. La prépondérance du nucléaire dans la production d’électricité ne doit pas occulter la dépendance de la France aux énergies fossiles et importées pour ses besoins en énergie. Dès lors, l’atteinte de la neutralité carbone oblige à renoncer en quasi-totalité à ces énergies fossiles."

docs = [{"content": text1}, {"content": text2}, {"content": text3}]

# Initialize document store and write in the documents
document_store = ElasticsearchDocumentStore()
document_store.write_documents(docs)

# Load machine translation models
from haystack.nodes import TransformersTranslator

in_translator = TransformersTranslator(model_name_or_path="Helsinki-NLP/opus-mt-fr-en")
out_translator = TransformersTranslator(model_name_or_path="Helsinki-NLP/opus-mt-en-fr")

# Wrap the previously defined QuestionAnswerGenerationPipeline
from haystack.pipelines import TranslationWrapperPipeline

pipeline_with_translation = TranslationWrapperPipeline(
    input_translator=in_translator, output_translator=out_translator, pipeline=qag_pipeline
)

for idx, document in enumerate(tqdm(document_store)):
    print(f"\n * Generating questions and answers for document {idx}: {document.content[:100]}...\n")
    result = pipeline_with_translation.run(documents=[document])
    print_questions(result)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
0it [00:00, ?it/s]


 * Generating questions and answers for document 0: En France, environ 60% de l’énergie utilisée est d’origine fossile : il s’agit principalement des pr...



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3415: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior


Generated pairs:
 - Q:Quel pourcentage de l'énergie française est d'origine fossile?
      A: 60%
 - Q:Quelle est la principale source d'énergie en France?
      A: produits pétroliers
 - Q:Quelle est la quantité d'énergie que représentent principalement les produits pétroliers?
      A: 40 % environ),
 - Q:Où la France importe-t-elle son énergie?
      A: pays producteurs
 - Q:L'Arabie saoudite et l'Algérie pour le pétrole brut, la Norvège, la Russie, les Pays-Bas et le Nigéria pour le gaz?
      A: importations en provenance des pays producteurs

 * Generating questions and answers for document 1: C’est pourtant bien ce système articulé autour des énergies fossiles qui a été le socle de la croiss...




Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 23.19 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 23.05 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 23.69 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 23.16 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 22.84 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 22.86 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 23.21 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 21.94 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 23.42 Batches/s]
2it [00:14,  7.58s/it]


Generated pairs:
 - Q:Qu'est-ce qui a été fourni à la France malgré les chocs pétroliers?
      A: énergie bon marché
 - Q:Quelle est la consommation finale de combustibles fossiles par an?
      A: 930 TWh
 - Q:Quelle quantité d'électricité la France utilise-t-elle?
      A: 430 TWh
 - Q:Sur combien de réacteurs nucléaires la centrale nucléaire compte-t-elle?
      A: Annexe V. Partie I
 - Q:Quand les réacteurs ont-ils été construits et mis en service?
      A: entre la fin des années 1970 et le début des années 90
 - Q:Combien de réacteurs y a-t-il dans le système?
      A: Annexe V. Partie I
 - Q:Quand le programme nucléaire français a-t-il répondu à un souci d'autonomie énergétique?
      A: à la suite de chocs pétroliers
 - Q:Quel est le principal atout de la France dans la lutte contre le changement climatique?
      A: Le programme nucléaire français
 - Q:Qu'est-ce que la France produit qui est largement décarboné en grandes quantités?
      A: électricité

 * Generating questi


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 24.56 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 24.51 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 24.19 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 22.84 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 22.77 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 22.92 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 23.43 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 22.19 Batches/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 24.42 Batches/s]
3it [00:23,  7.69s/it]


Generated pairs:
 - Q:Quel a été le sujet d'un débat démocratique en France?
      A: le choix de l'énergie nucléaire civile
 - Q:Qu'est-ce que la discussion s'est cristallisée autour de la notion de "partie" de l'énergie nucléaire?
      A: mélange d'électricité
 - Q:Quel pourcentage de l'électricité produite en France est l'énergie nucléaire?
      A: 70 %
 - Q:Quelle part de l'énergie finale de la France est nucléaire?
      A: moins de 20 %
 - Q:L'énergie nucléaire représente quel pourcentage de l'électricité produite?
      A: 70 %
 - Q:Quel pourcentage de l'énergie finale utilisée par les Français est nucléaire?
      A: 20%
 - Q:Qu'est-ce que la prépondérance de l'énergie nucléaire dans la production d'électricité n'est pas obscure?
      A: La dépendance de la France à l'égard des combustibles fossiles
 - Q:Qu'est-ce qui exige que presque toutes ces énergies fossiles soient abandonnées?
      A: la réalisation de la neutralité en matière de carbone;
 - Q:Qu'exige la neutralité